In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets  
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import  DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold , cross_val_score
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier

In [2]:
# import some data to play with
fraud_df = pd.read_csv('D:/Work/Data Science and Analyst Course/ExcelR/Data Science/Assignments/14_Decision Trees/Fraud_check.csv',index_col=0)

In [3]:
fraud_df

,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
Undergrad,,,,,
NO,Single,68833,50047,10,YES
YES,Divorced,33700,134075,18,YES
NO,Married,36925,160205,30,YES
YES,Single,50190,193264,15,YES
NO,Married,81002,27533,28,NO
...,...,...,...,...,...
YES,Divorced,76340,39492,7,YES
YES,Divorced,69967,55369,2,YES
NO,Divorced,47334,154058,0,YES


In [4]:
df_fraud=fraud_df.reset_index()

In [5]:
df_fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [6]:
df_fraud.isnull().sum()

Undergrad          0
Marital.Status     0
Taxable.Income     0
City.Population    0
Work.Experience    0
Urban              0
dtype: int64

In [7]:
df_fraud.columns

Index(['Undergrad', 'Marital.Status', 'Taxable.Income', 'City.Population',
       'Work.Experience', 'Urban'],
      dtype='object')

### Treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

In [8]:
df_fraud["income"]="<=30000"

In [9]:
df_fraud.loc[df_fraud["Taxable.Income"]>=30000,"income"]="Good"
df_fraud.loc[df_fraud["Taxable.Income"]<=30000,"income"]="Risky"

In [10]:
df_fraud["income"].unique()

array(['Good', 'Risky'], dtype=object)

### Renaming Some Columns

In [11]:
df_fraud.rename(columns={'Marital.Status':'Marital_Status','City.Population':'City_Population','Work.Experience':'Work_Experience','Taxable.Income':'Taxable_Income'},inplace=True)

In [12]:
df_fraud

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban,income
0,NO,Single,68833,50047,10,YES,Good
1,YES,Divorced,33700,134075,18,YES,Good
2,NO,Married,36925,160205,30,YES,Good
3,YES,Single,50190,193264,15,YES,Good
4,NO,Married,81002,27533,28,NO,Good
...,...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES,Good
596,YES,Divorced,69967,55369,2,YES,Good
597,NO,Divorced,47334,154058,0,YES,Good
598,YES,Married,98592,180083,17,NO,Good


In [13]:
colnames=list(df_fraud.columns)

In [14]:
colnames

['Undergrad',
 'Marital_Status',
 'Taxable_Income',
 'City_Population',
 'Work_Experience',
 'Urban',
 'income']

### Label Encoding for categorical data

In [15]:
le=LabelEncoder()
select_columns=["Undergrad","Marital_Status","Urban"]
le.fit(df_fraud[select_columns].values.flatten())
df_fraud[select_columns]=df_fraud[select_columns].apply(le.fit_transform)

In [16]:
df_fraud

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban,income
0,0,2,68833,50047,10,1,Good
1,1,0,33700,134075,18,1,Good
2,0,1,36925,160205,30,1,Good
3,1,2,50190,193264,15,1,Good
4,0,1,81002,27533,28,0,Good
...,...,...,...,...,...,...,...
595,1,0,76340,39492,7,1,Good
596,1,0,69967,55369,2,1,Good
597,0,0,47334,154058,0,1,Good
598,1,1,98592,180083,17,0,Good


In [18]:
df_fraud.shape

(600, 7)

In [21]:
df_fraud.income.value_counts()

Good     476
Risky    124
Name: income, dtype: int64

In [22]:
X=df_fraud.drop('income',axis=1)
Y=df_fraud['income']

In [23]:
X

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
0,0,2,68833,50047,10,1
1,1,0,33700,134075,18,1
2,0,1,36925,160205,30,1
3,1,2,50190,193264,15,1
4,0,1,81002,27533,28,0
...,...,...,...,...,...,...
595,1,0,76340,39492,7,1
596,1,0,69967,55369,2,1
597,0,0,47334,154058,0,1
598,1,1,98592,180083,17,0


In [24]:
Y

0      Good
1      Good
2      Good
3      Good
4      Good
       ... 
595    Good
596    Good
597    Good
598    Good
599    Good
Name: income, Length: 600, dtype: object

### Random Forest Classifier

In [27]:
rf = RandomForestClassifier(n_jobs=3,oob_score=True,n_estimators=100,criterion="entropy")
rf

RandomForestClassifier(criterion='entropy', n_jobs=3, oob_score=True)

In [28]:
rf.fit(X,Y)

RandomForestClassifier(criterion='entropy', n_jobs=3, oob_score=True)

In [29]:
rf.classes_

array(['Good', 'Risky'], dtype=object)

In [30]:
rf.n_outputs_

1

In [31]:
rf.n_features_

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


6

In [32]:
rf.oob_score_

0.9983333333333333

In [33]:
rf.predict(X)

array(['Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Risky', 'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Risky', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Risky', 'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Risky',
       'Good', 'Good', 'Risky', 'Good', 'Good', 'Risky', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Risky',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Risky',
       'Good', 'Risky', 'Good', 'Good', 'Good', 'Risky', 'Risky', 'Good',
       'Risky', 'Good', 'Risky', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Good', 'Good'

In [35]:
df_fraud['pred'] = rf.predict(X)

In [36]:
cols = ['pred','income']
cols

['pred', 'income']

In [37]:
df_fraud[cols]

,pred,income
0,Good,Good
1,Good,Good
2,Good,Good
3,Good,Good
4,Good,Good
...,...,...
595,Good,Good
596,Good,Good
597,Good,Good
598,Good,Good


In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df_fraud['income'],df_fraud['pred'])

array([[476,   0],
       [  0, 124]], dtype=int64)

In [41]:
pd.crosstab(df_fraud['income'],df_fraud['pred'])

pred,Good,Risky
income,,
Good,476,0
Risky,0,124
